# graph._utils.graph_spec_to_mermaid

> TODO fill in description

In [ ]:
#| default_exp graph._utils.graph_spec_to_mermaid

In [ ]:
#| hide
from nbdev.showdoc import *;

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
from __future__ import annotations
from IPython.display import Markdown
from typing import List

import fbdev
from fbdev.comp.port import PortType, PortSpec, PortSpecCollection
from fbdev.comp import BaseComponent
from fbdev.graph.graph_spec import GraphSpec, NodePortSpec, NodeSpec

In [ ]:
from fbdev.graph._utils.graph_spec_to_mermaid import graph_to_mermaid

`__graph_to_mermaid_subgraph_structs` convers a graph into a list of Mermaid subgraph 'structs', each represented a node. If a node contains an fbdev subgraph, it will nest subgraphs to it recursively.

*Note: A Mermaid and an fbdev subgraph are different things.*

In [ ]:
#|exporti
def __is_port_connected(port:NodePortSpec) -> bool:
    # This is really just used in `fbdev.graph.graph_spec_to_mermaid.graph_to_mermaid`
    externally_connected = (port._port_type, port.name) in port._parent_node._edge_connections
    if type(port._parent_node) == NodeSpec and port._parent_node.contains_graph:
        subgraph = port._parent_node.subgraph
        internally_connected = (port._port_type, port.name) in subgraph._edge_connections
    else: internally_connected = False
    return externally_connected or internally_connected

In [ ]:
#|exporti
__port_type_to_node_shape = {
    PortType.INPUT: '[%s]',
    PortType.CONFIG: '[(%s)]',
    PortType.SIGNAL: '>%s]',
    PortType.OUTPUT: '[%s]',
    PortType.MESSAGE: '[[%s]]',
}

def __get_port_address(address_prefix:str, port:NodePortSpec):
    return f"{address_prefix}{port._parent_id}:{port.id_str}"
def __get_port_label(port:NodePortSpec):
    return __port_type_to_node_shape[port.port_type] % port.name
def __add_to_mm_class(mm_class, mm_id, mm_class_assignments):
    if mm_class not in mm_class_assignments:
        mm_class_assignments[mm_class] = []
    mm_class_assignments[mm_class].append(mm_id)

def __graph_to_mermaid_subgraph_structs(address_prefix:str,
                                        graph:GraphSpec,
                                        mm_edges,
                                        mm_class_assignments,
                                        hide_unconnected_ports:bool,
                                        hide_port_types:List[PortType]):
    def get_mermaid_struct():
        return {
            'mm_address' : None,
            'mm_label' : None,
            'mm_nodes' : [],
            'mm_node_labels' : [],
            'mm_subgraphs' : [],
        }
        
    def get_mm_edge(edge):
        tail_address = __get_port_address(address_prefix, edge.tail_port)
        head_address = __get_port_address(address_prefix, edge.head_port)
        arrow = '-.->' if edge.tail_is_graph or edge.head_is_graph else '-->'
        if edge.is_maxsize_finite: arrow = f"{arrow}|{edge.maxsize}|"
        return tail_address, arrow, head_address

    mm_subgraphs = []

    # Turn graph child nodes into mermaid subgraphs
    for node_spec in graph.nodes.values():
        mm_node_spec = get_mermaid_struct()
        mm_node_spec['mm_address'] = f"{address_prefix}{node_spec.id}"
        mm_node_spec['mm_label'] = node_spec.rich_id
        # Ports
        for port in node_spec.ports.iter_ports():
            if hide_unconnected_ports and not __is_port_connected(port): continue
            if not __is_port_connected(port) and port.port_type in hide_port_types: continue
            port_address = __get_port_address(address_prefix, port)
            __add_to_mm_class(port.port_type.label, port_address, mm_class_assignments)
            mm_node_spec['mm_nodes'].append(port_address)
            mm_node_spec['mm_node_labels'].append(__get_port_label(port))
        # Edges
        for edge in node_spec.edge_connections.values():
            if edge.tail is not None and edge.head is not None:
                mm_edges.add(get_mm_edge(edge))
        # Recursively deal with subgraphs
        if node_spec.contains_graph:
            # Add child nodes
            mm_node_spec_children_container = get_mermaid_struct()
            mm_node_spec_children_container['mm_address'] = f"{address_prefix}{node_spec.id}__PROP__children"
            __add_to_mm_class('subgraph_zone', mm_node_spec_children_container['mm_address'], mm_class_assignments)
            mm_node_spec_children_container['mm_label'] = " "
            mm_node_spec_children_container['mm_subgraphs'] = __graph_to_mermaid_subgraph_structs(
                f"{mm_node_spec['mm_address']}.", node_spec.subgraph, mm_edges, mm_class_assignments,
                hide_unconnected_ports, hide_port_types
            )
            mm_node_spec['mm_subgraphs'].append(mm_node_spec_children_container)
            
        mm_subgraphs.append(mm_node_spec)
        
    # Graph edge connections
    for edge_id in graph._edge_connections.values():
        edge = graph.edges[edge_id]
        if edge.tail is not None and edge.head is not None:
            mm_edges.add(get_mm_edge(edge))
        
    return mm_subgraphs

`__convert_mm_structs_to_mermaid` converts the mermaid subgraph structs into a mermaid flowchart.

In [ ]:
#|exporti
__invalid_mm_character_conversions = {
    '.' : '__D__',
    ':' : '__C__',
}

def __fmt(txt):
    return ''.join([__invalid_mm_character_conversions[c] if c in __invalid_mm_character_conversions else c for c in txt])

def __convert_mm_structs_to_mermaid(merm_lines, mm_subgraph_structs, tab_level):
    def add_line(add_tabs, l="", format=True):
        if format: l = __fmt(l)
        merm_lines.append((tab_level+add_tabs) * '    ' + l)
        
    for mm_sgs in mm_subgraph_structs:
        add_line(0, f"subgraph {mm_sgs['mm_address']}[\"{mm_sgs['mm_label']}\"]")
        for mm_node, mm_node_label in zip(mm_sgs['mm_nodes'], mm_sgs['mm_node_labels']):
            add_line(1, f"{mm_node}{mm_node_label}")
        __convert_mm_structs_to_mermaid(merm_lines, mm_sgs['mm_subgraphs'], tab_level+1)
        add_line(0, 'end')

In [ ]:
#|export
__mermaid_class_defs = {
    'input' : 'fill:#13543e',
    'output' : 'fill:#0d1b59',
    'subgraph_zone' : 'fill:#000',
}

In [ ]:
#|hide
show_doc(graph_to_mermaid)

---

### graph_to_mermaid

>      graph_to_mermaid (graph:fbdev.graph.graph_spec.GraphSpec, orientation='',
>                        hide_unconnected_ports=False,
>                        hide_port_types:List[fbdev.comp.port.PortType]=[])

*TB - Top to bottom
TD - Top-down/ same as top to bottom
BT - Bottom to top
RL - Right to left
LR - Left to right*

In [ ]:
#|export
def graph_to_mermaid(graph:GraphSpec,
                     orientation='',
                     hide_unconnected_ports=False,
                     hide_port_types:List[PortType]=[]) -> str:
    """
    TB - Top to bottom
    TD - Top-down/ same as top to bottom
    BT - Bottom to top
    RL - Right to left
    LR - Left to right
    """
    mm_class_assignments = {}
    mm_edges = set()
    
    mm_subgraph_structs = __graph_to_mermaid_subgraph_structs("", graph, mm_edges, mm_class_assignments,
                                                              hide_unconnected_ports, hide_port_types)
    merm_lines = [f'flowchart {orientation}']
    __convert_mm_structs_to_mermaid(merm_lines, mm_subgraph_structs, 1)
    
    # Add graph ports
    for port in graph.ports.iter_ports():
        port_address = __get_port_address('', port)
        __add_to_mm_class(port.port_type.label, port_address, mm_class_assignments)
        merm_lines.append(__fmt(f"    {port_address}{__get_port_label(port)}"))

    # Add edges
    for mm_edge in mm_edges:
        tail_address, arrow, head_address = mm_edge
        tail_address = __fmt(tail_address.replace(f".{GraphSpec.GRAPH_ID}", '')) # A hack...
        head_address = __fmt(head_address.replace(f".{GraphSpec.GRAPH_ID}", ''))
        merm_lines.append(f"    {tail_address} {arrow} {head_address}")
        
    # Add class definitions
    for class_name, class_def in __mermaid_class_defs.items():
        merm_lines.append(f"    classDef {__fmt(class_name)} {class_def};")
        
    # Add classes
    for class_name, class_instances in mm_class_assignments.items():
        class_instances = ",".join(class_instances)
        merm_lines.append(__fmt(f"    class {class_instances} {class_name};"))
        
    return "\n".join(merm_lines)

In [ ]:
class FooComponent1(BaseComponent):
    port_specs = PortSpecCollection(
        PortSpec(PortType.INPUT, "inp"),
        PortSpec(PortType.OUTPUT, "out1"),
        PortSpec(PortType.OUTPUT, "out2"),
        PortSpec(PortType.CONFIG, "conf1"),
    )
    
class FooComponent2(BaseComponent):
    port_specs = PortSpecCollection(
        PortSpec(PortType.INPUT, "inp1"),
        PortSpec(PortType.INPUT, "inp2"),
        PortSpec(PortType.OUTPUT, "out1"),
        PortSpec(PortType.OUTPUT, "out2"),
        PortSpec(PortType.SIGNAL, "sig1"),
    )
    
class FooComponent3(BaseComponent):
    port_specs = PortSpecCollection(
        PortSpec(PortType.INPUT, "inp1"),
        PortSpec(PortType.INPUT, "inp2"),
        PortSpec(PortType.OUTPUT, "out"),
    )

graph = GraphSpec(PortSpecCollection())

graph.add_graph_port(PortSpec(PortType.INPUT, "inp"))
graph.add_graph_port(PortSpec(PortType.OUTPUT, "out"))

node1 = graph.add_node(FooComponent1)
node2 = graph.add_node(FooComponent2)
node3 = graph.add_node(FooComponent3)

graph.ports.input.inp >> node1.ports.input.inp
node1.ports.output.out1 >> node2.ports.input.inp1
node2.ports.input.inp2 << graph.add_edge(maxsize=5) << node1.ports.output.out2
node2.ports.output.out1 >> node3.ports.input.inp1
node2.ports.output.out2 >> node3.ports.input.inp2
node3.ports.output.out >> graph.add_edge(maxsize=5) >> graph.ports.output.out

GRAPH:output.out

In [ ]:
Markdown(f"```mermaid\n{graph_to_mermaid(graph)}\n```")

```mermaid
flowchart 
    subgraph FooComponent1["FooComponent1[]"]
        FooComponent1__C__input__D__inp[inp]
        FooComponent1__C__output__D__out1[out1]
        FooComponent1__C__output__D__out2[out2]
        FooComponent1__C__config__D__conf1[(conf1)]
        FooComponent1__C__message__D__started[[started]]
        FooComponent1__C__message__D__terminated[[terminated]]
    end
    subgraph FooComponent2["FooComponent2[]"]
        FooComponent2__C__input__D__inp1[inp1]
        FooComponent2__C__input__D__inp2[inp2]
        FooComponent2__C__output__D__out1[out1]
        FooComponent2__C__output__D__out2[out2]
        FooComponent2__C__signal__D__sig1>sig1]
        FooComponent2__C__message__D__started[[started]]
        FooComponent2__C__message__D__terminated[[terminated]]
    end
    subgraph FooComponent3["FooComponent3[]"]
        FooComponent3__C__input__D__inp1[inp1]
        FooComponent3__C__input__D__inp2[inp2]
        FooComponent3__C__output__D__out[out]
        FooComponent3__C__message__D__started[[started]]
        FooComponent3__C__message__D__terminated[[terminated]]
    end
    GRAPH__C__input__D__inp[inp]
    GRAPH__C__output__D__out[out]
    GRAPH__C__input__D__inp -.-> FooComponent1__C__input__D__inp
    FooComponent1__C__output__D__out2 -->|5| FooComponent2__C__input__D__inp2
    FooComponent1__C__output__D__out1 --> FooComponent2__C__input__D__inp1
    FooComponent2__C__output__D__out1 --> FooComponent3__C__input__D__inp1
    FooComponent3__C__output__D__out -.->|5| GRAPH__C__output__D__out
    FooComponent2__C__output__D__out2 --> FooComponent3__C__input__D__inp2
    classDef input fill:#13543e;
    classDef output fill:#0d1b59;
    classDef subgraph_zone fill:#000;
    class FooComponent1__C__input__D__inp,FooComponent2__C__input__D__inp1,FooComponent2__C__input__D__inp2,FooComponent3__C__input__D__inp1,FooComponent3__C__input__D__inp2,GRAPH__C__input__D__inp input;
    class FooComponent1__C__output__D__out1,FooComponent1__C__output__D__out2,FooComponent2__C__output__D__out1,FooComponent2__C__output__D__out2,FooComponent3__C__output__D__out,GRAPH__C__output__D__out output;
    class FooComponent1__C__config__D__conf1 config;
    class FooComponent1__C__message__D__started,FooComponent1__C__message__D__terminated,FooComponent2__C__message__D__started,FooComponent2__C__message__D__terminated,FooComponent3__C__message__D__started,FooComponent3__C__message__D__terminated message;
    class FooComponent2__C__signal__D__sig1 signal;
```

In [ ]:
Markdown(f"```mermaid\n{graph_to_mermaid(graph, hide_unconnected_ports=True)}\n```")

```mermaid
flowchart 
    subgraph FooComponent1["FooComponent1[]"]
        FooComponent1__C__input__D__inp[inp]
        FooComponent1__C__output__D__out1[out1]
        FooComponent1__C__output__D__out2[out2]
    end
    subgraph FooComponent2["FooComponent2[]"]
        FooComponent2__C__input__D__inp1[inp1]
        FooComponent2__C__input__D__inp2[inp2]
        FooComponent2__C__output__D__out1[out1]
        FooComponent2__C__output__D__out2[out2]
    end
    subgraph FooComponent3["FooComponent3[]"]
        FooComponent3__C__input__D__inp1[inp1]
        FooComponent3__C__input__D__inp2[inp2]
        FooComponent3__C__output__D__out[out]
    end
    GRAPH__C__input__D__inp[inp]
    GRAPH__C__output__D__out[out]
    GRAPH__C__input__D__inp -.-> FooComponent1__C__input__D__inp
    FooComponent1__C__output__D__out2 -->|5| FooComponent2__C__input__D__inp2
    FooComponent1__C__output__D__out1 --> FooComponent2__C__input__D__inp1
    FooComponent2__C__output__D__out1 --> FooComponent3__C__input__D__inp1
    FooComponent3__C__output__D__out -.->|5| GRAPH__C__output__D__out
    FooComponent2__C__output__D__out2 --> FooComponent3__C__input__D__inp2
    classDef input fill:#13543e;
    classDef output fill:#0d1b59;
    classDef subgraph_zone fill:#000;
    class FooComponent1__C__input__D__inp,FooComponent2__C__input__D__inp1,FooComponent2__C__input__D__inp2,FooComponent3__C__input__D__inp1,FooComponent3__C__input__D__inp2,GRAPH__C__input__D__inp input;
    class FooComponent1__C__output__D__out1,FooComponent1__C__output__D__out2,FooComponent2__C__output__D__out1,FooComponent2__C__output__D__out2,FooComponent3__C__output__D__out,GRAPH__C__output__D__out output;
```

In [ ]:
Markdown(f"```mermaid\n{graph_to_mermaid(graph, hide_port_types=[PortType.SIGNAL, PortType.MESSAGE])}\n```")

```mermaid
flowchart 
    subgraph FooComponent1["FooComponent1[]"]
        FooComponent1__C__input__D__inp[inp]
        FooComponent1__C__output__D__out1[out1]
        FooComponent1__C__output__D__out2[out2]
        FooComponent1__C__config__D__conf1[(conf1)]
    end
    subgraph FooComponent2["FooComponent2[]"]
        FooComponent2__C__input__D__inp1[inp1]
        FooComponent2__C__input__D__inp2[inp2]
        FooComponent2__C__output__D__out1[out1]
        FooComponent2__C__output__D__out2[out2]
    end
    subgraph FooComponent3["FooComponent3[]"]
        FooComponent3__C__input__D__inp1[inp1]
        FooComponent3__C__input__D__inp2[inp2]
        FooComponent3__C__output__D__out[out]
    end
    GRAPH__C__input__D__inp[inp]
    GRAPH__C__output__D__out[out]
    GRAPH__C__input__D__inp -.-> FooComponent1__C__input__D__inp
    FooComponent1__C__output__D__out2 -->|5| FooComponent2__C__input__D__inp2
    FooComponent1__C__output__D__out1 --> FooComponent2__C__input__D__inp1
    FooComponent2__C__output__D__out1 --> FooComponent3__C__input__D__inp1
    FooComponent3__C__output__D__out -.->|5| GRAPH__C__output__D__out
    FooComponent2__C__output__D__out2 --> FooComponent3__C__input__D__inp2
    classDef input fill:#13543e;
    classDef output fill:#0d1b59;
    classDef subgraph_zone fill:#000;
    class FooComponent1__C__input__D__inp,FooComponent2__C__input__D__inp1,FooComponent2__C__input__D__inp2,FooComponent3__C__input__D__inp1,FooComponent3__C__input__D__inp2,GRAPH__C__input__D__inp input;
    class FooComponent1__C__output__D__out1,FooComponent1__C__output__D__out2,FooComponent2__C__output__D__out1,FooComponent2__C__output__D__out2,FooComponent3__C__output__D__out,GRAPH__C__output__D__out output;
    class FooComponent1__C__config__D__conf1 config;
```

In [ ]:
Markdown(f"```mermaid\n{graph_to_mermaid(graph, hide_port_types=[PortType.INPUT])}\n```")

```mermaid
flowchart 
    subgraph FooComponent1["FooComponent1[]"]
        FooComponent1__C__input__D__inp[inp]
        FooComponent1__C__output__D__out1[out1]
        FooComponent1__C__output__D__out2[out2]
        FooComponent1__C__config__D__conf1[(conf1)]
        FooComponent1__C__message__D__started[[started]]
        FooComponent1__C__message__D__terminated[[terminated]]
    end
    subgraph FooComponent2["FooComponent2[]"]
        FooComponent2__C__input__D__inp1[inp1]
        FooComponent2__C__input__D__inp2[inp2]
        FooComponent2__C__output__D__out1[out1]
        FooComponent2__C__output__D__out2[out2]
        FooComponent2__C__signal__D__sig1>sig1]
        FooComponent2__C__message__D__started[[started]]
        FooComponent2__C__message__D__terminated[[terminated]]
    end
    subgraph FooComponent3["FooComponent3[]"]
        FooComponent3__C__input__D__inp1[inp1]
        FooComponent3__C__input__D__inp2[inp2]
        FooComponent3__C__output__D__out[out]
        FooComponent3__C__message__D__started[[started]]
        FooComponent3__C__message__D__terminated[[terminated]]
    end
    GRAPH__C__input__D__inp[inp]
    GRAPH__C__output__D__out[out]
    GRAPH__C__input__D__inp -.-> FooComponent1__C__input__D__inp
    FooComponent1__C__output__D__out2 -->|5| FooComponent2__C__input__D__inp2
    FooComponent1__C__output__D__out1 --> FooComponent2__C__input__D__inp1
    FooComponent2__C__output__D__out1 --> FooComponent3__C__input__D__inp1
    FooComponent3__C__output__D__out -.->|5| GRAPH__C__output__D__out
    FooComponent2__C__output__D__out2 --> FooComponent3__C__input__D__inp2
    classDef input fill:#13543e;
    classDef output fill:#0d1b59;
    classDef subgraph_zone fill:#000;
    class FooComponent1__C__input__D__inp,FooComponent2__C__input__D__inp1,FooComponent2__C__input__D__inp2,FooComponent3__C__input__D__inp1,FooComponent3__C__input__D__inp2,GRAPH__C__input__D__inp input;
    class FooComponent1__C__output__D__out1,FooComponent1__C__output__D__out2,FooComponent2__C__output__D__out1,FooComponent2__C__output__D__out2,FooComponent3__C__output__D__out,GRAPH__C__output__D__out output;
    class FooComponent1__C__config__D__conf1 config;
    class FooComponent1__C__message__D__started,FooComponent1__C__message__D__terminated,FooComponent2__C__message__D__started,FooComponent2__C__message__D__terminated,FooComponent3__C__message__D__started,FooComponent3__C__message__D__terminated message;
    class FooComponent2__C__signal__D__sig1 signal;
```

In [ ]:
top_graph = GraphSpec(PortSpecCollection(
    PortSpec(PortType.INPUT, "inp"),
    PortSpec(PortType.OUTPUT, "out")
))

graph_node = top_graph.add_node(fbdev.graph.GraphComponentFactory.create_component(graph))
top_graph.ports.input.inp >> graph_node.ports.input.inp
top_graph.ports.output.out << graph_node.ports.output.out

GraphComponent:output.out

In [ ]:
Markdown(f"```mermaid\n{graph_to_mermaid(top_graph, hide_unconnected_ports=True)}\n```")

```mermaid
flowchart 
    subgraph GraphComponent["GraphComponent[]"]
        GraphComponent__C__input__D__inp[inp]
        GraphComponent__C__output__D__out[out]
        subgraph GraphComponent__PROP__children[" "]
            subgraph GraphComponent__D__FooComponent1["FooComponent1[]"]
                GraphComponent__D__FooComponent1__C__input__D__inp[inp]
                GraphComponent__D__FooComponent1__C__output__D__out1[out1]
                GraphComponent__D__FooComponent1__C__output__D__out2[out2]
            end
            subgraph GraphComponent__D__FooComponent2["FooComponent2[]"]
                GraphComponent__D__FooComponent2__C__input__D__inp1[inp1]
                GraphComponent__D__FooComponent2__C__input__D__inp2[inp2]
                GraphComponent__D__FooComponent2__C__output__D__out1[out1]
                GraphComponent__D__FooComponent2__C__output__D__out2[out2]
            end
            subgraph GraphComponent__D__FooComponent3["FooComponent3[]"]
                GraphComponent__D__FooComponent3__C__input__D__inp1[inp1]
                GraphComponent__D__FooComponent3__C__input__D__inp2[inp2]
                GraphComponent__D__FooComponent3__C__output__D__out[out]
            end
        end
    end
    GRAPH__C__input__D__inp[inp]
    GRAPH__C__output__D__out[out]
    GraphComponent__C__output__D__out -.-> GRAPH__C__output__D__out
    GraphComponent__D__FooComponent1__C__output__D__out2 -->|5| GraphComponent__D__FooComponent2__C__input__D__inp2
    GRAPH__C__input__D__inp -.-> GraphComponent__C__input__D__inp
    GraphComponent__D__FooComponent1__C__output__D__out1 --> GraphComponent__D__FooComponent2__C__input__D__inp1
    GraphComponent__D__FooComponent2__C__output__D__out1 --> GraphComponent__D__FooComponent3__C__input__D__inp1
    GraphComponent__D__FooComponent2__C__output__D__out2 --> GraphComponent__D__FooComponent3__C__input__D__inp2
    GraphComponent__C__input__D__inp -.-> GraphComponent__D__FooComponent1__C__input__D__inp
    GraphComponent__D__FooComponent3__C__output__D__out -.->|5| GraphComponent__C__output__D__out
    classDef input fill:#13543e;
    classDef output fill:#0d1b59;
    classDef subgraph_zone fill:#000;
    class GraphComponent__C__input__D__inp,GraphComponent__D__FooComponent1__C__input__D__inp,GraphComponent__D__FooComponent2__C__input__D__inp1,GraphComponent__D__FooComponent2__C__input__D__inp2,GraphComponent__D__FooComponent3__C__input__D__inp1,GraphComponent__D__FooComponent3__C__input__D__inp2,GRAPH__C__input__D__inp input;
    class GraphComponent__C__output__D__out,GraphComponent__D__FooComponent1__C__output__D__out1,GraphComponent__D__FooComponent1__C__output__D__out2,GraphComponent__D__FooComponent2__C__output__D__out1,GraphComponent__D__FooComponent2__C__output__D__out2,GraphComponent__D__FooComponent3__C__output__D__out,GRAPH__C__output__D__out output;
    class GraphComponent__PROP__children subgraph_zone;
```

In [ ]:
Markdown(f"```mermaid\n{graph_to_mermaid(top_graph, hide_unconnected_ports=False)}\n```")

```mermaid
flowchart 
    subgraph GraphComponent["GraphComponent[]"]
        GraphComponent__C__input__D__inp[inp]
        GraphComponent__C__output__D__out[out]
        GraphComponent__C__message__D__started[[started]]
        GraphComponent__C__message__D__terminated[[terminated]]
        subgraph GraphComponent__PROP__children[" "]
            subgraph GraphComponent__D__FooComponent1["FooComponent1[]"]
                GraphComponent__D__FooComponent1__C__input__D__inp[inp]
                GraphComponent__D__FooComponent1__C__output__D__out1[out1]
                GraphComponent__D__FooComponent1__C__output__D__out2[out2]
                GraphComponent__D__FooComponent1__C__config__D__conf1[(conf1)]
                GraphComponent__D__FooComponent1__C__message__D__started[[started]]
                GraphComponent__D__FooComponent1__C__message__D__terminated[[terminated]]
            end
            subgraph GraphComponent__D__FooComponent2["FooComponent2[]"]
                GraphComponent__D__FooComponent2__C__input__D__inp1[inp1]
                GraphComponent__D__FooComponent2__C__input__D__inp2[inp2]
                GraphComponent__D__FooComponent2__C__output__D__out1[out1]
                GraphComponent__D__FooComponent2__C__output__D__out2[out2]
                GraphComponent__D__FooComponent2__C__signal__D__sig1>sig1]
                GraphComponent__D__FooComponent2__C__message__D__started[[started]]
                GraphComponent__D__FooComponent2__C__message__D__terminated[[terminated]]
            end
            subgraph GraphComponent__D__FooComponent3["FooComponent3[]"]
                GraphComponent__D__FooComponent3__C__input__D__inp1[inp1]
                GraphComponent__D__FooComponent3__C__input__D__inp2[inp2]
                GraphComponent__D__FooComponent3__C__output__D__out[out]
                GraphComponent__D__FooComponent3__C__message__D__started[[started]]
                GraphComponent__D__FooComponent3__C__message__D__terminated[[terminated]]
            end
        end
    end
    GRAPH__C__input__D__inp[inp]
    GRAPH__C__output__D__out[out]
    GraphComponent__C__output__D__out -.-> GRAPH__C__output__D__out
    GraphComponent__D__FooComponent1__C__output__D__out2 -->|5| GraphComponent__D__FooComponent2__C__input__D__inp2
    GRAPH__C__input__D__inp -.-> GraphComponent__C__input__D__inp
    GraphComponent__D__FooComponent1__C__output__D__out1 --> GraphComponent__D__FooComponent2__C__input__D__inp1
    GraphComponent__D__FooComponent2__C__output__D__out1 --> GraphComponent__D__FooComponent3__C__input__D__inp1
    GraphComponent__D__FooComponent2__C__output__D__out2 --> GraphComponent__D__FooComponent3__C__input__D__inp2
    GraphComponent__C__input__D__inp -.-> GraphComponent__D__FooComponent1__C__input__D__inp
    GraphComponent__D__FooComponent3__C__output__D__out -.->|5| GraphComponent__C__output__D__out
    classDef input fill:#13543e;
    classDef output fill:#0d1b59;
    classDef subgraph_zone fill:#000;
    class GraphComponent__C__input__D__inp,GraphComponent__D__FooComponent1__C__input__D__inp,GraphComponent__D__FooComponent2__C__input__D__inp1,GraphComponent__D__FooComponent2__C__input__D__inp2,GraphComponent__D__FooComponent3__C__input__D__inp1,GraphComponent__D__FooComponent3__C__input__D__inp2,GRAPH__C__input__D__inp input;
    class GraphComponent__C__output__D__out,GraphComponent__D__FooComponent1__C__output__D__out1,GraphComponent__D__FooComponent1__C__output__D__out2,GraphComponent__D__FooComponent2__C__output__D__out1,GraphComponent__D__FooComponent2__C__output__D__out2,GraphComponent__D__FooComponent3__C__output__D__out,GRAPH__C__output__D__out output;
    class GraphComponent__C__message__D__started,GraphComponent__C__message__D__terminated,GraphComponent__D__FooComponent1__C__message__D__started,GraphComponent__D__FooComponent1__C__message__D__terminated,GraphComponent__D__FooComponent2__C__message__D__started,GraphComponent__D__FooComponent2__C__message__D__terminated,GraphComponent__D__FooComponent3__C__message__D__started,GraphComponent__D__FooComponent3__C__message__D__terminated message;
    class GraphComponent__PROP__children subgraph_zone;
    class GraphComponent__D__FooComponent1__C__config__D__conf1 config;
    class GraphComponent__D__FooComponent2__C__signal__D__sig1 signal;
```